In [25]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI
import json
import requests
import pandas as pd
import numpy

# Load environment variables from the .env file
load_dotenv()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None) 


In [20]:
# Database file path
database_file_path = "./db/test.db"

# Create the SQLAlchemy engine
engine = create_engine(f'sqlite:///{database_file_path}')

@tool
def calculate_price(product_list: list) -> float:
    """
    Calculates the total price of a list of products.

    Args:
        product_list (list): A list of dictionaries representing products.
        
    Returns:
        float: The total price of the products.
    """
    total_price = sum(product['price'] for product in product_list)
    return total_price

@tool
def get_product_price(product_name: str) -> str:
    """
    Retrieves the price of a specific product.

    Args:
        product_name (str): The name of the product.
        
    Returns:
        str: The price of the product in JSON format.
    """
    try:
        query = text("""
            SELECT name, price
            FROM Product
            WHERE name LIKE :product_name;
        """)

        with engine.connect() as connection:
            result = pd.read_sql_query(query, connection, params={"product_name": f"%{product_name}%"})
        
        if not result.empty:
            return json.dumps(result.to_dict('records')[0])
        else:
            return json.dumps({"product_name": product_name, "price": None})
    except Exception as e:
        return json.dumps({"error": str(e)})

@tool
def get_products_available() -> str:
    """
    Retrieves the list of products that are available.

    Returns:
        str: A list of available products in JSON format.
    """
    try: 
        query = text("""
            SELECT name, price
            FROM Product
            WHERE available = 1;
        """)

        with engine.connect() as connection:
            result = pd.read_sql_query(query, connection)
        
        if not result.empty:
            return json.dumps(result.to_dict('records'))
        else:
            return json.dumps({"available_products": []})
    except Exception as e:
        return json.dumps({"error": str(e)})

# Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Define the tools
tools_sql = [get_product_price, get_products_available, calculate_price]

# Get the API key from environment variables
api_key = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key)

# Create the agent
agent = create_tool_calling_agent(llm, tools_sql, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools_sql, verbose=True)

# Example usage
# agent_executor.invoke({"input": "What is the total cost for a portable charger and a keyboard?"})


## Tools / API calls

### Token

In [21]:
base_url = 'https://dashboard.colorines.paitesting.com/api'

# Set the URL for the token API
url_token = base_url+"/v1/token"

# Set the data for the POST request
data = {
    "email": "sale@platform.com",
    "password": os.getenv('CONSULTANT_PASSWORD'),
    "device_name": "colorina_2",
    "role": "consultant"
}

# Send a POST request
response = requests.post(url_token, json=data)

#print(response)

# Extract the token from the response
token = response.json().get('data').get('token')

#print("Token:", token)


<Response [200]>
Token: 2105|ZiWHTKW9ISsH4gI6DIgBUJm2voUzH8OZhTdKOdXp1670a56f


### Components

In [44]:
import requests
import pandas as pd
import json

#TODO: API need to add a filter by 'is_active'

def GetProperties(token: str):
    headers = {"Authorization": f"Bearer {token}"}
    url = "https://dashboard.colorines.paitesting.com/api/v1/properties"
    params = {'include': 'step,project,development'} 
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        # Check if the response contains any content
        if response.content:
            data = response.json()
        else:
            return pd.DataFrame({"error": ["Empty response from the server"]})
        
    except requests.exceptions.RequestException as e:
        return pd.DataFrame({"error": [str(e)]})
    except ValueError as ve:
        return pd.DataFrame({"error": ["Error parsing JSON response"]})
    
    # Check if the 'data' key exists and contains a list of dictionaries
    if 'data' in data:
        if isinstance(data['data'], list):
            return pd.DataFrame(data['data'])
        else:
            return pd.DataFrame({"error": ["'data' key does not contain a list"]})
    else:
        return pd.DataFrame({"error": ["'data' key not found in the response"]})

#PREPROCESS Properties
def PreprocessProperties(df):
    df['project_name'] = df['project'].apply(lambda x: x.get('name'))
    df['development_name'] = df['development'].apply(lambda x: x.get('name'))
    df['step_number'] = df['step'].apply(lambda x: x.get('step_number')) #numero de etapa
    return df



# Example usage

df_properties = GetProperties(token)

def GetProjects(token: str):
    headers = {"Authorization": f"Bearer {token}"}
    url = "https://dashboard.colorines.paitesting.com/api/v1/projects"
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()
    except requests.exceptions.RequestException as e:
        return pd.DataFrame({"error": [str(e)]})
    
    # Check if the 'data' key exists and contains a list of dictionaries
    if 'data' in data:
        if isinstance(data['data'], list):
            return pd.DataFrame(data['data'])
        else:
            return pd.DataFrame({"error": ["'data' key does not contain a list"]})
    else:
        return pd.DataFrame({"error": ["'data' key not found in the response"]})

# Example usage

#df_projects = GetProjects(token)

def GetDevelopments(token: str):
    headers = {"Authorization": f"Bearer {token}"}
    url = "https://dashboard.colorines.paitesting.com/api/v1/developments"
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()
    except requests.exceptions.RequestException as e:
        return pd.DataFrame({"error": [str(e)]})
    
    # Check if the 'data' key exists and contains a list of dictionaries
    if 'data' in data:
        if isinstance(data['data'], list):
            return pd.DataFrame(data['data'])
        else:
            return pd.DataFrame({"error": ["'data' key does not contain a list"]})
    else:
        return pd.DataFrame({"error": ["'data' key not found in the response"]})

# Example usage

#df_devs = GetDevelopments(token)



In [57]:
463.81 * 1400

649334.0

In [62]:
df_properties

,id,custom_id,alias,square_foot,status,prices,svg_url,step,project,development,is_active,customer_id,locked_until,project_id,project_name,development_name,step_number
0,356,p1,TR-04-001,"{'default_value': 463.81, 'with_decimals': '463.81', 'without_decimals': '463'}","{'name': 'Apartado', 'short_code': 'booked', 'is_available': False}","[{'type': 'property', 'amount': 1400, 'amount_with_decimals': '1,400.00', 'amount_without_decimals': '1400', 'total_amount': 649334, 'total_amount_with_decimals': '649,334.00', 'total_amount_without_decimals': '649334', 'currency_id': 1, 'currency_symbol': '$', 'currency_code': 'MXN'}, {'type': 'reserve', 'amount': 3500, 'amount_with_decimals': '3,500.00', 'amount_without_decimals': '3500', 'total_amount': 3500, 'total_amount_with_decimals': '3,500.00', 'total_amount_without_decimals': '3500', 'currency_id': 1, 'currency_symbol': '$', 'currency_code': 'MXN'}, {'type': 'property', 'amount': 84, 'amount_with_decimals': '84.00', 'amount_without_decimals': '84', 'total_amount': 38960.04, 'total_amount_with_decimals': '38,960.04', 'total_amount_without_decimals': '38960', 'currency_id': 2, 'currency_symbol': '$', 'currency_code': 'USD'}, {'type': 'reserve', 'amount': 210, 'amount_with_decimals': '210.00', 'amount_without_decimals': '210', 'total_amount': 210, 'total_amount_with_decimals': '210.00', 'total_amount_without_decimals': '210', 'currency_id': 2, 'currency_symbol': '$', 'currency_code': 'USD'}]",https://dashboard.colorines.paitesting.com/storage/media/16/TRES-RAICES-2_E4_SVG-(Color-blanco-y-bordes-negros).svg,"{'id': 1, 'step_number': 4, 'down_payment': {'symbol': '%', 'min': 0, 'default': 30, 'max': 100, 'max_installments': 5, 'installments_percent': 30}, 'months': {'symbol': None, 'min': 24, 'default': 24, 'max': 72}, 'contract_type': {'id': 2, 'name': 'Venta a plazos'}, 'sqft_prices': [{'type': 'property', 'amount': 1400, 'amount_with_decimals': '1,400.00', 'amount_without_decimals': '1400', 'total_amount': 1400, 'total_amount_with_decimals': '1,400.00', 'total_amount_without_decimals': '1400', 'currency_id': 1, 'currency_symbol': '$', 'currency_code': 'MXN'}, {'type': 'reserve', 'amount': 3500, 'amount_with_decimals': '3,500.00', 'amount_without_decimals': '3500', 'total_amount': 3500, 'total_amount_with_decimals': '3,500.00', 'total_amount_without_decimals': '3500', 'currency_id': 1, 'currency_symbol': '$', 'currency_code': 'MXN'}, {'type': 'property', 'amount': 84, 'amount_with_decimals': '84.00', 'amount_without_decimals': '84', 'total_amount': 84, 'total_amount_with_decimals': '84.00', 'total_amount_without_decimals': '84', 'currency_id': 2, 'currency_symbol': '$', 'currency_code': 'USD'}, {'type': 'reserve', 'amount': 210, 'amount_with_decimals': '210.00', 'amount_without_decimals': '210', 'total_amount': 210, 'total_amount_with_decimals': '210.00', 'total_amount_without_decimals': '210', 'currency_id': 2, 'currency_symbol': '$', 'currency_code': 'USD'}], 'is_active': True, 'is_progressive': False, 'optional_discount_percent': 10, 'svg_url': 'https://dashboard.colorines.paitesting.com/storage/media/16/TRES-RAICES-2_E4_SVG-(Color-blanco-y-bordes-negros).svg', 'prices': [{'type': 'property', 'amount': 1400, 'amount_with_decimals': '1,400.00', 'amount_without_decimals': '1400', 'total_amount': 1400, 'total_amount_with_decimals': '1,400.00', 'total_amount_without_decimals': '1400', 'currency_id': 1, 'currency_symbol': '$', 'currency_code': 'MXN'}, {'type': 'reserve', 'amount': 3500, 'amount_with_decimals': '3,500.00', 'amount_without_decimals': '3500', 'total_amount': 3500, 'total_amount_with_decimals': '3,500.00', 'total_amount_without_decimals': '3500', 'currency_id': 1, 'currency_symbol': '$', 'currency_code': 'MXN'}, {'type': 'property', 'amount': 84, 'amount_with_decimals': '84.00', 'amount_without_decimals': '84', 'total_amount': 84, 'total_amount_with_decimals': '84.00', 'total_amount_without_decimals': '84', 'currency_id': 2, 'currency_symbol': '$', 'currency_code': 'USD'}, {'t

In [77]:
# Extracting 'id' from 'project' using Pandas


df_properties['project_name'] = df_properties['project'].apply(lambda x: x.get('name'))
df_properties['development_name'] = df_properties['development'].apply(lambda x: x.get('name'))
df_properties['step_number'] = df_properties['step'].apply(lambda x: x.get('step_number')) #numero de etapa

def extract_total_amount(prices, currency_code):
    total_amount = None  # Initialize to None or any default value as needed
    for entry in prices:
        if entry['type'] == 'property' and entry['currency_code'] == currency_code:
            total_amount = entry['total_amount']
            break  # Exit loop once found
    return total_amount

# Extracting total_amount_mxn and total_amount_usd
df_properties['total_amount_mxn'] = df_properties['prices'].apply(lambda x: extract_total_amount(x, 'MXN'))
df_properties['total_amount_usd'] = df_properties['prices'].apply(lambda x: extract_total_amount(x, 'USD'))

df_properties

,id,custom_id,alias,square_foot,status,prices,svg_url,step,project,development,is_active,customer_id,locked_until,project_id,project_name,development_name,step_number,total_amount_mxn,total_amount_usd
0,356,p1,TR-04-001,"{'default_value': 463.81, 'with_decimals': '463.81', 'without_decimals': '463'}","{'name': 'Apartado', 'short_code': 'booked', 'is_available': False}","[{'type': 'property', 'amount': 1400, 'amount_with_decimals': '1,400.00', 'amount_without_decimals': '1400', 'total_amount': 649334, 'total_amount_with_decimals': '649,334.00', 'total_amount_without_decimals': '649334', 'currency_id': 1, 'currency_symbol': '$', 'currency_code': 'MXN'}, {'type': 'reserve', 'amount': 3500, 'amount_with_decimals': '3,500.00', 'amount_without_decimals': '3500', 'total_amount': 3500, 'total_amount_with_decimals': '3,500.00', 'total_amount_without_decimals': '3500', 'currency_id': 1, 'currency_symbol': '$', 'currency_code': 'MXN'}, {'type': 'property', 'amount': 84, 'amount_with_decimals': '84.00', 'amount_without_decimals': '84', 'total_amount': 38960.04, 'total_amount_with_decimals': '38,960.04', 'total_amount_without_decimals': '38960', 'currency_id': 2, 'currency_symbol': '$', 'currency_code': 'USD'}, {'type': 'reserve', 'amount': 210, 'amount_with_decimals': '210.00', 'amount_without_decimals': '210', 'total_amount': 210, 'total_amount_with_decimals': '210.00', 'total_amount_without_decimals': '210', 'currency_id': 2, 'currency_symbol': '$', 'currency_code': 'USD'}]",https://dashboard.colorines.paitesting.com/storage/media/16/TRES-RAICES-2_E4_SVG-(Color-blanco-y-bordes-negros).svg,"{'id': 1, 'step_number': 4, 'down_payment': {'symbol': '%', 'min': 0, 'default': 30, 'max': 100, 'max_installments': 5, 'installments_percent': 30}, 'months': {'symbol': None, 'min': 24, 'default': 24, 'max': 72}, 'contract_type': {'id': 2, 'name': 'Venta a plazos'}, 'sqft_prices': [{'type': 'property', 'amount': 1400, 'amount_with_decimals': '1,400.00', 'amount_without_decimals': '1400', 'total_amount': 1400, 'total_amount_with_decimals': '1,400.00', 'total_amount_without_decimals': '1400', 'currency_id': 1, 'currency_symbol': '$', 'currency_code': 'MXN'}, {'type': 'reserve', 'amount': 3500, 'amount_with_decimals': '3,500.00', 'amount_without_decimals': '3500', 'total_amount': 3500, 'total_amount_with_decimals': '3,500.00', 'total_amount_without_decimals': '3500', 'currency_id': 1, 'currency_symbol': '$', 'currency_code': 'MXN'}, {'type': 'property', 'amount': 84, 'amount_with_decimals': '84.00', 'amount_without_decimals': '84', 'total_amount': 84, 'total_amount_with_decimals': '84.00', 'total_amount_without_decimals': '84', 'currency_id': 2, 'currency_symbol': '$', 'currency_code': 'USD'}, {'type': 'reserve', 'amount': 210, 'amount_with_decimals': '210.00', 'amount_without_decimals': '210', 'total_amount': 210, 'total_amount_with_decimals': '210.00', 'total_amount_without_decimals': '210', 'currency_id': 2, 'currency_symbol': '$', 'currency_code': 'USD'}], 'is_active': True, 'is_progressive': False, 'optional_discount_percent': 10, 'svg_url': 'https://dashboard.colorines.paitesting.com/storage/media/16/TRES-RAICES-2_E4_SVG-(Color-blanco-y-bordes-negros).svg', 'prices': [{'type': 'property', 'amount': 1400, 'amount_with_decimals': '1,400.00', 'amount_without_decimals': '1400', 'total_amount': 1400, 'total_amount_with_decimals': '1,400.00', 'total_amount_without_decimals': '1400', 'currency_id': 1, 'currency_symbol': '$', 'currency_code': 'MXN'}, {'type': 'reserve', 'amount': 3500, 'amount_with_decimals': '3,500.00', 'amount_without_decimals': '3500', 'total_amount': 3500, 'total_amount_with_decimals': '3,500.00', 'total_amount_without_decimals': '3500', 'currency_id': 1, 'currency_symbol': '$', 'currency_code': 'MXN'}, {'type': 'property', 'amount': 84, 'amount_with_decimals': '84.00', 'amount_without_decimals': '84', 'total_amount': 84, 'total_amount_with_decimals': '84.00', 'total_amount_without_decimals': '84', 'currency_id': 2, 'currency_symbol':

In [74]:
type(df_properties['prices'].iloc[0][4])

IndexError: list index out of range

In [33]:
df_devs

,id,name,is_active,country,logo_url,cover_url
0,4,Yucatán,True,"{'id': 155, 'iso': 'MX', 'icons': {'flag': '🇲🇽'}, 'name': {'en': 'United Mexican States', 'es': 'México'}}",https://dashboard.colorines.paitesting.com/storage/media/11/PHOTO-2024-05-27-11-14-17.jpg,https://dashboard.colorines.paitesting.com/storage/media/12/PHOTO-2024-05-27-11-14-17.jpg


In [31]:
df_projects

,id,name,prefix,is_active,logo_url,cover_url
0,3,Tres Raíces,TR,True,https://dashboard.colorines.paitesting.com/storage/media/13/PHOTO-2024-05-27-11-14-17.jpg,https://dashboard.colorines.paitesting.com/storage/media/14/PHOTO-2024-05-27-11-14-17.jpg


In [28]:
properties_df = df_properties[df_properties['is_active'] == True]

In [29]:
properties_df

,id,custom_id,alias,square_foot,status,prices,svg_url,step,is_active,customer_id,locked_until
41,397,p42,TR-04-042,"{'default_value': 300, 'with_decimals': '300.00', 'without_decimals': '300'}","{'name': 'Disponible', 'short_code': 'available', 'is_available': True}","[{'type': 'property', 'amount': 1400, 'amount_with_decimals': '1,400.00', 'amount_without_decimals': '1400', 'total_amount': 420000, 'total_amount_with_decimals': '420,000.00', 'total_amount_without_decimals': '420000', 'currency_id': 1, 'currency_symbol': '$', 'currency_code': 'MXN'}, {'type': 'reserve', 'amount': 3500, 'amount_with_decimals': '3,500.00', 'amount_without_decimals': '3500', 'total_amount': 3500, 'total_amount_with_decimals': '3,500.00', 'total_amount_without_decimals': '3500', 'currency_id': 1, 'currency_symbol': '$', 'currency_code': 'MXN'}, {'type': 'property', 'amount': 84, 'amount_with_decimals': '84.00', 'amount_without_decimals': '84', 'total_amount': 25200, 'total_amount_with_decimals': '25,200.00', 'total_amount_without_decimals': '25200', 'currency_id': 2, 'currency_symbol': '$', 'currency_code': 'USD'}, {'type': 'reserve', 'amount': 210, 'amount_with_decimals': '210.00', 'amount_without_decimals': '210', 'total_amount': 210, 'total_amount_with_decimals': '210.00', 'total_amount_without_decimals': '210', 'currency_id': 2, 'currency_symbol': '$', 'currency_code': 'USD'}]",https://dashboard.colorines.paitesting.com/storage/media/16/TRES-RAICES-2_E4_SVG-(Color-blanco-y-bordes-negros).svg,"{'id': 1, 'step_number': 4, 'down_payment': {'symbol': '%', 'min': 0, 'default': 30, 'max': 100, 'max_installments': 5, 'installments_percent': 30}, 'months': {'symbol': None, 'min': 24, 'default': 24, 'max': 72}, 'contract_type': {'id': 2, 'name': 'Venta a plazos'}, 'sqft_prices': [{'type': 'property', 'amount': 1400, 'amount_with_decimals': '1,400.00', 'amount_without_decimals': '1400', 'total_amount': 1400, 'total_amount_with_decimals': '1,400.00', 'total_amount_without_decimals': '1400', 'currency_id': 1, 'currency_symbol': '$', 'currency_code': 'MXN'}, {'type': 'reserve', 'amount': 3500, 'amount_with_decimals': '3,500.00', 'amount_without_decimals': '3500', 'total_amount': 3500, 'total_amount_with_decimals': '3,500.00', 'total_amount_without_decimals': '3500', 'currency_id': 1, 'currency_symbol': '$', 'currency_code': 'MXN'}, {'type': 'property', 'amount': 84, 'amount_with_decimals': '84.00', 'amount_without_decimals': '84', 'total_amount': 84, 'total_amount_with_decimals': '84.00', 'total_amount_without_decimals': '84', 'currency_id': 2, 'currency_symbol': '$', 'currency_code': 'USD'}, {'type': 'reserve', 'amount': 210, 'amount_with_decimals': '210.00', 'amount_without_decimals': '210', 'total_amount': 210, 'total_amount_with_decimals': '210.00', 'total_amount_without_decimals': '210', 'currency_id': 2, 'currency_symbol': '$', 'currency_code': 'USD'}], 'is_active': True, 'is_progressive': False, 'optional_discount_percent': 10, 'svg_url': 'https://dashboard.colorines.paitesting.com/storage/media/16/TRES-RAICES-2_E4_SVG-(Color-blanco-y-bordes-negros).svg', 'prices': [{'type': 'property', 'amount': 1400, 'amount_with_decimals': '1,400.00', 'amount_without_decimals': '1400', 'total_amount': 1400, 'total_amount_with_decimals': '1,400.00', 'total_amount_without_decimals': '1400', 'currency_id': 1, 'currency_symbol': '$', 'currency_code': 'MXN'}, {'type': 'reserve', 'amount': 3500, 'amount_with_decimals': '3,500.00', 'amount_without_decimals': '3500', 'total_amount': 3500, 'total_amount_with_decimals': '3,500.00', 'total_amount_without_decimals': '3500', 'currency_id': 1, 'currency_symbol': '$', 'currency_code': 'MXN'}, {'type': 'property', 'amount': 84, 'amount_with_decimals': '84.00', 'amount_without_decimals': '84', 'total_amount': 84, 'total_amount_with_decimals': '84.00', 'total_amount_without_decimals': '84', 'currency_id': 2, 'currency_symbol': '$', 'currency_code': 'USD'}, {'type': 'reserve', 'amount': 210, 'amount_with_decimals': '210.00', 'amount

## Tools

In [12]:
from langchain_core.tools import tool
import requests
import json

import requests
import json

@tool
def get_active_properties(token: str) -> str:
    """
    Uses the token to fetch the active properties from colorines API.
    
    Args:
        token (str): The authorization token required to access the API.

    Returns:
        str: The JSON response from the API as a string.
    """
    properties_df = GetProperties(token)
    properties_df = properties_df[properties_df['is_active'] == True]
    

    response = requests.get(url, headers=headers, params=params)
    # Check if the response was successful
    if response.status_code == 200:
        try:
            return json.dumps(response.json())  # Convert JSON response to string
        except ValueError:  # Catch JSON errors
            return json.dumps({"error": "Error parsing JSON"})
    else:
        return json.dumps({"error": f"Received status code {response.status_code}"})



@tool
def get_last_sale(token: str) -> str:
    """
    Uses the token to fetch the last sale from colorines API.
    
    Args:
        token (str): The authorization token required to access the API.

    Returns:
        str: The JSON response from the API as a string.
    """
    headers = {"Authorization": f"Bearer {token}"}
    url = "https://dashboard.colorines.paitesting.com/api/v1/sales"
    params = {"per_page": 1, "sort": "-created_at"}
    
    try:
        response = requests.get(url, headers=headers, params=params)
        # Check if the response was successful
        if response.status_code == 200:
            try:
                return json.dumps(response.json())  # Convert JSON response to string
            except ValueError:  # Catch JSON errors
                return json.dumps({"error": "Error parsing JSON"})
        else:
            return json.dumps({"error": f"Received status code {response.status_code}"})
    except requests.exceptions.RequestException as e:
        return json.dumps({"error": str(e)})

#TODO: Receive customer ID and filter the information before passing it

@tool
def get_last_sale(token: str) -> str:
    """
    Uses the token to fetch the last sale from colorines API.
    
    Args:
        token (str): The authorization token required to access the API.

    Returns:
        str: The JSON response from the API as a string.
    """
    headers = {"Authorization": f"Bearer {token}"}
    url = "https://dashboard.colorines.paitesting.com/api/v1/sales"
    params = {"per_page": 1, "sort": "-created_at"}
    
    try:
        response = requests.get(url, headers=headers, params=params)
        # Check if the response was successful
        if response.status_code == 200:
            try:
                return json.dumps(response.json())  # Convert JSON response to string
            except ValueError:  # Catch JSON errors
                return json.dumps({"error": "Error parsing JSON"})
        else:
            return json.dumps({"error": f"Received status code {response.status_code}"})
    except requests.exceptions.RequestException as e:
        return json.dumps({"error": str(e)})




In [ ]:
    
# Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant who will use the {token} to use the tools to bring information to the user"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Example of integrating with langchain agent framework
api_key = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key)

# Add the `get_sales` function to the list of tools
tools_sql.append(get_last_sale)  # Assuming 'tools_sql' already exists and includes other tools

# Create the agent
agent = create_tool_calling_agent(llm, tools_sql, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools_sql, verbose=True)

# Example usage:
# token = 'your_actual_token_here'
# base_url = 'https://api.yourdomain.com'


In [ ]:
get_last_sale(token)

In [14]:
input_data = {
    "input": "dame información de mi última venta",
    "token": token
}

# Invoke the agent
agent_executor.invoke(input_data)



> Entering new AgentExecutor chain...

Invoking: `get_last_sale` with `{'token': '2089|1NtyvkQjbMInB89NF2aPBKEsBKiASG6iroNPTHAhdfa22064'}`


{"data": [{"id": 55, "custom_id": "sa_4HKug50MakfSTRruEU", "channel": "Intranet", "currency": "MXN", "reserve_amount": 3500, "property_amount": 517916, "property_discount_amount": 51791.6, "down_payment_amount": 186449, "down_payment_percent": 40, "down_payment_installments": 5, "number_installments": 24, "installments_amount": 279675.4, "installments_individual_amount": 11653, "sqft_price_amount": 1260, "status": {"name": "Contrato firmado", "short_code": "contract", "percent": 75}, "absolute_value": 466124.4, "amount_total": 466124.4, "amount_paid": 0, "amount_remaining": 466124.4, "first_payment_date": "2024-07-02T06:00:00.000000Z", "recurring_payday": 2, "next_payment_date": "2024-07-02T06:00:00.000000Z", "prices": [{"type": "property", "amount": 1400, "amount_with_decimals": "1,400.00", "amount_without_decimals": "1400", "total_amount": 140

{'input': 'dame información de mi última venta',
 'token': '2089|1NtyvkQjbMInB89NF2aPBKEsBKiASG6iroNPTHAhdfa22064',
 'output': 'La información de tu última venta es la siguiente:\n- ID de la venta: 55\n- ID personalizado: sa_4HKug50MakfSTRruEU\n- Canal de venta: Intranet\n- Moneda: MXN\n- Monto de la reserva: $3,500.00 MXN\n- Monto total de la propiedad: $517,916.00 MXN\n- Descuento en la propiedad: $51,791.60 MXN\n- Monto del enganche: $186,449.00 MXN\n- Porcentaje del enganche: 40%\n- Número de mensualidades: 24\n- Monto de las mensualidades: $279,675.40 MXN\n- Monto individual de las mensualidades: $11,653.00 MXN\n- Precio por metro cuadrado: $1,260.00 MXN\n- Estado de la venta: Contrato firmado\n- Fecha de la primera mensualidad: 02 de julio de 2024\n\n¡Si necesitas más detalles o información adicional, házmelo saber!'}

## Sales

In [ ]:
@tool
def get_customers(base_url: str, token: str) -> str:
    """
    Fetches a list of customers from Colorines API endpoint.

    Args:
    - base_url (str): The base URL of the API.
    - token (str): The authorization token required to access the API.

    Returns:
    - str: The JSON response from the API as a string.
    """
    headers = {
        "Authorization": f"Bearer {token}"
    }
    url_list_customers = f"{base_url}/v1/customers"
    response = requests.get(url_list_customers, headers=headers)
    return response.json()

# Example usage:
customers_data = get_customers(base_url, token)
customers_data

In [3]:
# Prueba la ejecución del agente
# agent_executor.invoke({"input": "Quiero comprar Un teclado un una camara cuanto me costaria"})

# Prueba de disponibilidad de productos
# agent_executor.invoke({"input": "Que productos tienes disponibles?"})
agent_executor.invoke({"input": "Quiero comprar un Cargador portátil y un teclado, cual es el total?"})



> Entering new AgentExecutor chain...

Invoking: `get_products_available` with `{}`


[{"name": "Teclado", "price": 75.25}, {"name": "Laptop", "price": 1000.0}, {"name": "Smartphone", "price": 800.0}, {"name": "Tablet", "price": 300.0}, {"name": "Impresora", "price": 200.0}, {"name": "Mano", "price": 150.0}, {"name": "C\u00e1mara", "price": 400.0}, {"name": "Televisor", "price": 700.0}, {"name": "Router", "price": 80.0}, {"name": "Reproductor Blu-ray", "price": 180.0}, {"name": "Teclado inal\u00e1mbrico", "price": 60.0}, {"name": "Mouse inal\u00e1mbrico", "price": 80.0}, {"name": "Webcam", "price": 70.0}, {"name": "Tarjeta de video", "price": 250.0}, {"name": "Memoria RAM", "price": 120.0}, {"name": "Disco duro externo", "price": 150.0}, {"name": "Tarjeta madre", "price": 350.0}, {"name": "Procesador", "price": 300.0}, {"name": "Gabinete para PC", "price": 80.0}, {"name": "Fuente de poder", "price": 100.0}, {"name": "Router inal\u00e1mbrico", "price": 50.0}, {"name": "Adaptador Wi-Fi

{'input': 'Quiero comprar un Cargador portátil y un teclado, cual es el total?',
 'output': 'El total para comprar un Cargador portátil y un teclado es de $115.25.'}